In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
import re

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
df = pd.read_csv('hotel-reviews.csv')

In [ ]:
df.head(2)

In [ ]:
a = pd.get_dummies(df.Is_Response,prefix='res')

In [ ]:
df_label = pd.concat([df,a],axis=1)

In [ ]:
df_label.head()

In [ ]:
df1 = df_label.drop(columns=['Is_Response','res_not happy'])

In [ ]:
df1.describe(include="O")

In [ ]:
def text_process(text):
#   word = text.lower()
#   word = word.split()
    word = re.sub('[^A-za-z]',' ',text)
    word = word.lower()
    word = word.split()
    word = [words for words in word if words not in stopwords.words('english')]
    word = [WordNetLemmatizer().lemmatize(words) for words in word]
    word = ' '.join(word)
    return word

In [ ]:
df1['Reviews'] = df1['Description'].iloc[:].apply(text_process)

In [ ]:
df1.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
cv = CountVectorizer()
X = cv.fit_transform(df1['Reviews'].iloc[:10000]).toarray()
# X = df1['Reviews']
y = df1['res_happy'].iloc[:10000]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [ ]:
cm

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))